In [1]:
# This is a new library compared to the previous modules. 
# Please perform "pip install sentence_transformers==2.7.0"
from sentence_transformers import SentenceTransformer

#sentence transformer helps in calling pretrained models from huggingface
model = SentenceTransformer("multi-qa-distilbert-cos-v1")

In [2]:
model.encode("I just discovered the course. Can I still join it?")

#Answer is 0.07

array([ 7.82226548e-02, -4.01311405e-02,  3.86135913e-02, -1.78966438e-04,
        8.92347097e-02, -5.04591092e-02, -1.05026569e-02,  3.71055678e-02,
       -4.18713912e-02,  3.48084792e-02, -1.20701883e-02, -2.36942340e-02,
        3.87900174e-02,  1.60988607e-02,  3.50747295e-02,  3.04746162e-03,
        5.79672381e-02, -4.10627462e-02, -3.41552682e-02, -2.56396383e-02,
       -3.55263911e-02,  1.42908087e-02, -1.62799917e-02,  3.21446545e-02,
       -4.66897376e-02,  7.89186060e-02,  4.90160920e-02,  1.56761166e-02,
       -1.69110075e-02,  2.26482227e-02,  5.60206100e-02, -3.98361087e-02,
        6.77409917e-02, -1.20209912e-02,  1.12621894e-03, -1.94394365e-02,
       -2.65951678e-02,  1.06177367e-02,  1.69687122e-02,  1.13487840e-02,
       -2.97063086e-02,  5.25258258e-02, -1.41453547e-02,  4.61699851e-02,
        1.17066065e-02, -2.38053519e-02, -6.32558241e-02, -1.92042235e-02,
       -7.10592186e-03,  3.24167833e-02,  2.49618199e-02, -5.27503015e-03,
        2.01149024e-02, -

Prepare the documents

In [3]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [4]:
# Filtering for 'machine-learning-zoomcamp' course
filtered_documents = [doc for doc in documents if doc['course'] == 'machine-learning-zoomcamp']

# Print the filtered documents
for doc in filtered_documents:
    print(doc)

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork', 'section': 'General course-related questions', 'question': 'How do I sign up?', 'course': 'machine-learning-zoomcamp', 'id': '0227b872'}
{'text': 'The course videos are pre-recorded, you can start watching the course right now.\nWe will also occasionally have office hours - live sessions where we will answer your questions. The office hours sessions are recorded too.\nYou can see the office hours as well as the pre-recorded course videos in the course playlist on YouTube.', 'section': 'General course-related questions', 'question': 'Is it going to be live? When?', 'course': 'ma

In [5]:
len(filtered_documents)

375


Q2. Creating the embeddings

In [6]:
# Create embeddings for each document
embeddings = []
for doc in filtered_documents:
    qa_text = f"{doc['question']} {doc['text']}"
    embedding = model.encode(qa_text)
    embeddings.append(embedding)

In [7]:
# Convert the list of embeddings to a NumPy array
import numpy as np
X = np.array(embeddings)

In [8]:
X.shape

#(375, 768)

(375, 768)

Q3. Search

In [9]:
# Example query
query = "I just discovered the course. Can I still join it?"

# Compute the embedding for the query
v = model.encode(query)

# Compute the cosine similarity scores
scores = X.dot(v)

# Print the scores
print(scores)

[ 2.89217263e-01  4.35050726e-01  2.20572561e-01  1.28085926e-01
  3.28754455e-01  4.58999664e-01  4.48930591e-01  2.70170599e-01
  2.13975310e-01  3.39745760e-01  1.63339168e-01  3.15918088e-01
  1.53621323e-02  4.14240420e-01  6.50657296e-01  4.12786484e-01
  2.02390462e-01  4.71707582e-01  3.74821007e-01  3.53319108e-01
  4.50558245e-01  1.85430944e-01  3.45526993e-01  8.22948217e-02
 -2.19077598e-02 -4.64250743e-02  8.60690773e-02  9.87197682e-02
 -2.14188285e-02  1.28751006e-02  2.54870541e-02  3.66866291e-02
  4.56338078e-02 -2.05279887e-02  1.82662532e-02  3.39789167e-02
  1.66510604e-03  1.03209512e-02  3.40170749e-02 -2.65231431e-02
 -4.11909968e-02 -4.97983992e-02 -6.20346665e-02 -7.63561390e-03
  4.61616889e-02  6.95944112e-03  4.77613509e-02 -9.97766852e-04
 -5.60126789e-02 -5.96206151e-02  2.05209196e-01 -1.16968602e-01
 -2.34160442e-02 -1.52456751e-02 -6.83515333e-03 -8.61715078e-02
  2.02636905e-02 -2.51591802e-02  4.21313569e-04 -3.79212126e-02
 -3.21442187e-02  2.48612

In [10]:
# Find and print the highest score
highest_score = np.max(scores)
print(f'The highest score is: {highest_score}')

#0.65

The highest score is: 0.6506572961807251


In [11]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filtered_documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

Q4. Hit-rate for our search engine

In [12]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [13]:
ground_truth

[{'question': 'Where can I sign up for the course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Can you provide a link to sign up?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Is there an FAQ for this Machine Learning course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Does this course have a GitHub repository for the sign-up link?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'How can I structure my questions and answers for the course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Are the course videos live or pre-recorded?',
  'course': 'machine-learning-zoomcamp',
  'document': '39fda9f0'},
 {'question': 'When can I start watching the course videos?',
  'course': 'machine-learning-zoomcamp',
  'document': '39fda9f0'},
 {'question': 'Are the live office hours sessions recorded?',
  'cours

In [14]:
# Calculate the hit rate
num_results = 5
hits = 0

for item in ground_truth:
    query = item['question']
    ground_truth_id = item['document']  # Using 'document' as the identifier
    
    # Compute the embedding for the query
    v_query = model.encode(query)
    
    # Get the top N results using the search engine
    results = search_engine.search(v_query, num_results=num_results)
    
    # Check if the ground truth id (document) is in the top N results
    result_ids = [res['id'] for res in results]
    if ground_truth_id in result_ids:
        hits += 1

# Calculate hit rate
hit_rate = hits / len(ground_truth)


In [15]:
hit_rate
#0.93

0.9398907103825137

Question 5:

In [18]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")
es_client.info()

ObjectApiResponse({'name': '5a97f49f913f', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'dtdtNPEySLC-OaX1GrlJeA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})